In [36]:
from itertools import permutations, product

import tqdm
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import os
from os import path
import numpy as np
import matplotlib.pyplot as plt

from warnings import simplefilter
from collections import OrderedDict

import pickle
import threading

if (os.path.abspath('').split('/')[-1] == 'project'):
    %cd utils
elif (os.path.abspath('').split('/')[-1] == 'train_and_vis'):
    %cd ../utils


import query_utils
import model_utils
import validation_utils
import data_utils
from StackingClassifier import StackingClassifier

print(os.path.abspath(''))

if (os.path.abspath('').split('/')[-1] == 'utils'):
    %cd ..


simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project


In [8]:
dataset = data_utils.retrieveMerged(dir='data/')

working directory: /Users/benjaminli/Documents/coding/scires/project


In [11]:
x = np.concatenate([dataset['merged']['normalized-4']['y_train'], dataset['merged']['normalized-4']['y_test']], axis=0).sum()
print(x)

1576


In [33]:
def assess_model(model, threshold, dataset):
    ds = dataset['f1-4']
    # mergedX = merged['X_test']
    # mergedY = merged['y_test']
    X, y = ds['X'], ds['y']

    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.2, random_state=42)

    bp = {
        'ans': {
            '0': {'1000': 0, '3000': 0, '5000': 0, '10000': 0, '>10000': 0},
            '1': {'1000': 0, '3000': 0, '5000': 0, '10000': 0, '>10000': 0},
        },
        'preds': {
            '0': {'1000': 0, '3000': 0, '5000': 0, '10000': 0, '>10000': 0},
            '1': {'1000': 0, '3000': 0, '5000': 0, '10000': 0, '>10000': 0},
        }
    }
    ds = dataset['f2-4']

    # mergedX = merged['X_test']
    # mergedY = merged['y_test']
    X, y = ds['X'], ds['y']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    threads = []

    if (type(model).__name__ == 'XGBClassifier'):
        X_test = X_test[model.get_booster().feature_names]

    for r in tqdm.tqdm(range(len(X_test))):
        data = pd.DataFrame([X_test.iloc[r].to_list()], columns=X_test.iloc[r].index)
        # print(data)
        pred_proba = model.predict_proba(data)[:,1]
        pred = '1' if pred_proba >= threshold else '0'
        s = X_test_reg.iloc[r].sum()
        res = str(y_test_reg[r])
        if s < 1000:
            if pred == res:
                bp['preds'][res]['1000']+=1
            bp['ans'][res]['1000']+=1
        elif s < 3000:
            if pred == res:
                bp['preds'][res]['3000']+=1
            bp['ans'][res]['3000']+=1
        elif s < 5000:
            if pred == res:
                bp['preds'][res]['5000']+=1
            bp['ans'][res]['5000']+=1
        elif s <= 10000:
            if pred == res:
                bp['preds'][res]['10000']+=1
            bp['ans'][res]['10000']+=1
        elif s > 10000:
            if pred == res:
                bp['preds'][res]['>10000']+=1
            bp['ans'][res]['>10000']+=1

    # for r in tqdm.tqdm(range(len(X_test))):
    #     data = pd.DataFrame([X_test.iloc[r].to_list()], columns=mergedXNormed.iloc[r].index)
    #     # print(data)
    #     pred_proba = model.predict_proba(data)[:,1]
    #     pred = 1 if pred_proba >= 0.5 else 0
    print(bp['ans'])
    print(bp['preds'])
    # print accuracies for each bp length
    zoo = 0
    zoo_total = 0
    nonzoo = 0
    nonzoo_total = 0
    for k in bp['ans']:
        # get overall accuracies for varying values of k
        for i in bp['ans'][k]:
            if k == '1':
                zoo += bp['preds'][k][i]
                zoo_total += bp['ans'][k][i]
            else:
                nonzoo += bp['preds'][k][i]
                nonzoo_total += bp['ans'][k][i]
            print(k, i, bp['preds'][k][i]/bp['ans'][k][i])
    
    print("zoonotic score", zoo/zoo_total)
    print("nonzoonotic score", nonzoo/nonzoo_total)
   

    # calculate f1 score
    y_pred = model.predict(X_test)
    print(f1_score(y_test, y_pred))


In [37]:
ds = dataset['f2-4']

X,y = ds['X'], ds['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model = pickle.load(open('models/curr_models/final_merged_stackingcv.pkl', 'rb'))
knn = pickle.load(open('models/curr_models/knn.pkl', 'rb'))
# rf = pickle.load(open('models/curr_models/randforest.pkl', 'rb'))
mlp = pickle.load(open('models/curr_models/mlpClassifier.pkl', 'rb')).best_estimator_
# xgboost = pickle.load(open('models/curr_models/xgBoost-f2-4-2.pkl', 'rb'))
svm = pickle.load(open('models/curr_models/svm.pkl', 'rb'))
em = pickle.load(open('models/curr_models/em-f2-4.pkl', 'rb'))

# print(f1_score(y_test_reg, em.predict(X_test_reg)))
# print(f1_score(y_test_reg, model.predict(X_test_reg)))
# print(f1_score(y_test_reg, knn.predict(X_test_reg)))
# print(f1_score(y_test_reg, rf.predict(X_test_reg)))
# print(f1_score(y_test_reg, mlp.predict(X_test_reg)))
# # print(f1_score(y_test_reg, xgboost.predict(X_test_reg)))
# print(f1_score(y_test_reg, svm.predict(X_test_reg)))
# print(f1_score(y_test_reg, xgbgrid.predict(X_test_reg)))


# assess_model(model=em, threshold=0.3, dataset=dataset)
# assess_model(model=model, threshold=0.5, dataset=dataset)

# make into threads
assess_model(em, 0.5, dataset)

# assess_model(model=em2, threshold=0.5, dataset=dataset)

100%|██████████| 2126/2126 [02:09<00:00, 16.40it/s]


{'0': {'1000': 19, '3000': 275, '5000': 189, '10000': 447, '>10000': 890}, '1': {'1000': 7, '3000': 31, '5000': 32, '10000': 173, '>10000': 63}}
{'0': {'1000': 19, '3000': 274, '5000': 185, '10000': 443, '>10000': 886}, '1': {'1000': 6, '3000': 23, '5000': 26, '10000': 167, '>10000': 60}}
0 1000 1.0
0 3000 0.9963636363636363
0 5000 0.9788359788359788
0 10000 0.9910514541387024
0 >10000 0.9955056179775281
1 1000 0.8571428571428571
1 3000 0.7419354838709677
1 5000 0.8125
1 10000 0.9653179190751445
1 >10000 0.9523809523809523
zoonotic score 0.9215686274509803
nonzoonotic score 0.9928571428571429
0.8342857142857144


In [13]:
# print(19/(19+335+217+642+923))
df = pd.DataFrame()
# fill df with the bp lengths as the index and the number of zoonotic and nonzoonotic as the columns
df['zoonotic'] = pd.Series(di_zoo)
df['pred_zoonotic'] = pd.Series(pred_di_zoo)
df['overall'] = pd.Series(di_overall)
df['pred_overall'] = pd.Series(pred_di_overall)
df['pred_overall%'] = df['pred_overall']/df['overall']

df.head()

,zoonotic,pred_zoonotic,overall,pred_overall,pred_overall%
1000,8,6,19,17,0.894737
3000,45,37,335,325,0.970149
5000,28,23,217,211,0.972350
10000,182,175,642,633,0.985981
>10000,66,60,923,913,0.989166
